### RQ1: How does an LLM-driven synthetic guideline affect diagnostic accuracy through clinical reasoning?

In [1]:
import os
import json
import pandas as pd

# Directory path containing JSONL files
directory = "results"

# Initialize an empty list to store data
data = []

# Read only files ending with 'eval_results-clf.jsonl' within the directory
for filename in filter(lambda f: f.endswith("eval_results-clf.jsonl"), os.listdir(directory)):
    filepath = os.path.join(directory, filename)
    
    # Extract model name (portion between 'snsb-' and '.eval_results.jsonl')
    test_type, *model_name_parts = filename.split('.eval_results')[0].split('-')
    model_name = "-".join(model_name_parts)
    
    with open(filepath, 'r') as file:
        for line in file:
            # Parse JSON data
            json_data = json.loads(line.strip())
            # Add extracted test type and model name to the data
            json_data['type'] = test_type
            json_data['model'] = model_name
            data.append(json_data)

# Convert the collected data into a DataFrame
df = pd.DataFrame(data)

# Convert accuracy, precision, sensitivity, specificity, and F1-score to percentages
for metric in ['accuracy', 'sensitivity', 'specificity', 'precision', 'f1_score']:
    df[metric] *= 100

# Mapping of model names for better readability
model_name_mapping = {
    "llama3.3:70b": "Llama 3.3 (70B)",
    "phi4:14b": "Phi 4 (14B)",
    "llama3.1:8b": "Llama 3.1 (8B)",
    "deepseek-r1:70b": "DeepSeek-R1 (70B)",
    "deepseek-r1:32b": "DeepSeek-R1 (32B)",
    "deepseek-r1:14b": "DeepSeek-R1 (14B)",
}
df['model'] = df['model'].replace(model_name_mapping)

# Specify the desired order of model names and test types
model_order = [
    "Llama 3.3 (70B)", "Phi 4 (14B)", "Llama 3.1 (8B)",
    "DeepSeek-R1 (70B)", "DeepSeek-R1 (32B)", "DeepSeek-R1 (14B)"
]
type_order = ["zeroshot", "fewshot1", "fewshot2", "custom"]

# Sort the DataFrame by model and test type
df['model'] = pd.Categorical(df['model'], categories=model_order, ordered=True)
df['type'] = pd.Categorical(df['type'], categories=type_order, ordered=True)

# Calculate average accuracy by model and test type
df_clf = df.groupby(['model', 'type'], observed=True).mean()

# Display relevant evaluation metrics
df_clf[['accuracy', 'sensitivity', 'specificity', 'precision', 'f1_score']]

accuracy  sensitivity  specificity  precision  \
model             type                                                      
Llama 3.3 (70B)   zeroshot     80.00       100.00        53.57      74.00   
                  fewshot1     83.08        91.89        71.43      80.95   
                  fewshot2     81.54        91.89        67.86      79.07   
                  custom       89.23        94.59        82.14      87.50   
Phi 4 (14B)       zeroshot     75.38        91.89        53.57      72.34   
                  fewshot1     72.31        83.78        57.14      72.09   
                  fewshot2     70.77        86.49        50.00      69.57   
                  custom       81.54        91.89        67.86      79.07   
Llama 3.1 (8B)    zeroshot     60.00        89.19        21.43      60.00   
                  fewshot1     70.77        97.30        35.71      66.67   
                  fewshot2     63.08        97.30        17.86      61.02   
                  custom       58.46        43.24        78.57      72.73   
DeepSeek-R1 (70B) zeroshot     73.85       100.00        19.05      72.13   
                  fewshot1     75.38        97.30        46.43      70.59   
                  fewshot2     69.23        94.59        35.71      66.04   
                  custom       60.00       100.00         7.14      58.73   
DeepSeek-R1 (32B) zeroshot     69.23       100.00         4.76      68.75   
                  fewshot1     78.46        91.89        60.71      75.56   
                  fewshot2     81.54        97.30        60.71      76.60   
                  custom       56.92       100.00         0.00      56.92   
DeepSeek-R1 (14B) zeroshot     80.00        90.91        57.14      81.63   
                  fewshot1     78.46        83.78        71.43      79.49   
                  fewshot2     72.31        81.08        60.71      73.17   
                  custom       73.85        72.97        75.00      79.41   

                            f1_score  
model             type                
Llama 3.3 (70B)   zeroshot     85.06  
                  fewshot1     86.08  
                  fewshot2     85.00  
                  custom       90.91  
Phi 4 (14B)       zeroshot     80.95  
                  fewshot1     77.50  
                  fewshot2     77.11  
                  custom       85.00  
Llama 3.1 (8B)    zeroshot     71.74  
                  fewshot1     79.12  
                  fewshot2     75.00  
                  custom       54.24  
DeepSeek-R1 (70B) zeroshot     83.81  
                  fewshot1     81.82  
                  fewshot2     77.78  
                  custom       74.00  
DeepSeek-R1 (32B) zeroshot     81.48  
                  fewshot1     82.93  
                  fewshot2     85.71  
                  custom       72.55  
DeepSeek-R1 (14B) zeroshot     86.02  
                  fewshot1     81.58  
                  fewshot2     76.92  
                  custom       76.06

### RQ2: How does an LLM-driven synthetic guideline impact on the quality of rationales?

In [2]:
import os
import pandas as pd
import json

# Directory containing JSONL files
directory = "results/"

# Initialize an empty list to store data
collected_data = []

# Iterate through files in the directory, processing only those ending with 'eval_results-rubric.jsonl'
for filename in os.listdir(directory):
    if filename.endswith("eval_results-rubric.jsonl"):  # Process only files matching the pattern
        filepath = os.path.join(directory, filename)
        
        # Extract model name and data type from the filename
        file_parts = filename.split('.eval_results')[0].split('-')
        data_type = file_parts[0]  # Extract the data type (first part)
        model_name = "-".join(file_parts[1:])  # Extract the model name (remaining parts)
        
        # Read and process the JSONL file
        with open(filepath, 'r') as file:
            for line in file:
                json_data = json.loads(line.strip())  # Parse JSON data
                json_data['data'] = data_type  # Add extracted data type
                json_data['model'] = model_name  # Add extracted model name
                collected_data.append(json_data)

# Convert collected data into a DataFrame
df = pd.DataFrame(collected_data)

# Mapping of model names for better readability
model_name_mapping = {
    "llama3.3:70b": "Llama 3.3 (70B)",
    "phi4:14b": "Phi 4 (14B)",
}
df['model'] = df['model'].replace(model_name_mapping)

score_mapping = {
    "A": "custom",
    "B": "fewshot"
}
df['score'] = df['score'].replace(score_mapping)

# Reorder the columns based on the specified order
criterion_order = ['consistency', 'correctness', 'specificity', 'helpfulness', 'humanlikeness']

# Sort the DataFrame by the specified order
df['criteria'] = pd.Categorical(df['criteria'], categories=criterion_order, ordered=True)

# Group data by 'model', and 'criteria', calculate normalized value counts of 'score'
result = df.groupby(['model', 'criteria'], observed=True)['score'].value_counts(normalize=True).unstack()

# Format the result as percentages
formatted_result = result.map(lambda x: f'{x:.2%}')

# Display the formatted DataFrame
formatted_result

score                          custom fewshot
model           criteria                     
Llama 3.3 (70B) consistency    83.08%  16.92%
                correctness    80.00%  20.00%
                specificity    86.15%  13.85%
                helpfulness    81.54%  18.46%
                humanlikeness  84.62%  15.38%
Phi 4 (14B)     consistency    66.15%  33.85%
                correctness    69.23%  30.77%
                specificity    76.92%  23.08%
                helpfulness    73.85%  26.15%
                humanlikeness  73.85%  26.15%